### Chord Extractor with librosa and chroma analysis
Testing triad detection and then 7th detection in the section of the chord to define the tetrad quality

In [2]:
import os
import librosa
import librosa.display
import numpy as np 
import importlib
import formExtractor as fem
importlib.reload(fem)
import matplotlib.pyplot as plt

In [3]:
def load_audio_files(path):
    audio_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".mp3"):
                audio_files.append(os.path.join(root, file))
    return audio_files

In [4]:
path = '/home/laura/aimir/'
collection = 'suno' #lastfm, suno, udio
song_files = path + collection + '/audio'
files = load_audio_files(song_files)
print(len(files))

96365


In [5]:
myFiles = files[:20]

In [6]:
id = 5
song = myFiles[id]

#song = '/home/laura/aimir/boomy/audio/14065870.mp3'
id_file = song.split('/')[-1].split('.')[0]
song

'/home/laura/aimir/suno/audio/0002d6bf-36e5-4f04-a075-5476f06b2c49.mp3'

In [7]:
song = "../src/test_audio/Djavan - Azul (Ao Vivo).wav"

In [8]:
#Get the tonality of the song

y, sr = librosa.load(song)

# Compute the chroma features using CQT
chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)

# Sum chroma features over time to emphasize prominent pitches
chroma_vector = np.sum(chroma_cq, axis=1)
chroma_vector /= np.linalg.norm(chroma_vector)

# Modified Krumhansl-Schmuckler key profiles
major_profile = np.array([6.35, 2.23, 3.48, 2.33, 4.38, 4.09,
                          2.52, 5.19, 2.39, 3.66, 2.29, 2.88])
minor_profile = np.array([6.33, 2.68, 3.52, 5.38, 2.60, 3.53,
                          2.54, 4.75, 3.98, 2.69, 3.34, 3.17])

# Optionally adjust profiles
# minor_profile[0] *= 1.2  # Emphasize tonic in minor

# Normalize the profiles
major_profile /= np.linalg.norm(major_profile)
minor_profile /= np.linalg.norm(minor_profile)

key_names = ['C', 'C#', 'D', 'D#', 'E', 'F',
             'F#', 'G', 'G#', 'A', 'A#', 'B']
correlations = []

for i in range(12):
    # Rotate the key profiles
    major_profile_rotated = np.roll(major_profile, i)
    minor_profile_rotated = np.roll(minor_profile, i)

    # Compute the correlation with adjusted weights
    major_corr = np.dot(chroma_vector, major_profile_rotated) * 0.9
    minor_corr = np.dot(chroma_vector, minor_profile_rotated) * 1.1  # Increase minor influence

    correlations.append({
        'key': key_names[i],
        'mode': 'major',
        'correlation': major_corr
    })
    correlations.append({
        'key': key_names[i],
        'mode': 'minor',
        'correlation': minor_corr
    })

# Find the best matching key
best_match = max(correlations, key=lambda x: x['correlation'])
tonality = f"{best_match['key']} {best_match['mode']}"
print(f"Tonality: {tonality}")

# Sort correlations in descending order
correlations_sorted = sorted(correlations, key=lambda x: x['correlation'], reverse=True)
print("Correlation scores for all keys:")
for corr in correlations_sorted:
    print(f"{corr['key']} {corr['mode']}: {corr['correlation']:.4f}")


Tonality: D minor
Correlation scores for all keys:
D minor: 1.0763
G minor: 1.0613
F minor: 1.0472
A minor: 1.0438
C minor: 1.0323
F# minor: 1.0278
E minor: 1.0264
A# minor: 1.0192
B minor: 1.0175
C# minor: 1.0076
D# minor: 0.9939
G# minor: 0.9889
F major: 0.8610
C major: 0.8556
D major: 0.8544
G major: 0.8536
A# major: 0.8478
A major: 0.8295
D# major: 0.8258
C# major: 0.8226
G# major: 0.8201
E major: 0.7970
F# major: 0.7952
B major: 0.7799


In [9]:
# Correct sharp or flat tonality
# Import your Transposition class
from transposition import Transposition

# Initialize the Transposition object
transposer = Transposition()

# Use the get_alterations_scales method to get alterations
corrected_tonality, alterations, scale = transposer.get_alterations_scales(tonality)

# Print the output
print(f"Tonality: {corrected_tonality}")
print(f"Alterations: {alterations}")
print(f"Scale: {scale}")

tonality = corrected_tonality


Tonality: D minor
Alterations: ['Bb']
Scale: ['D', 'E', 'F', 'G', 'A', 'Bb', 'C']


In [12]:
import triadExtractor as te
triads = te.TriadExtractor(hop_length=32, scale=scale)
chords = triads.extract_chords(song, window_duration=5, threshold=0.1, check_on_beat=True) #window_range minimum is 1
chords

[ChordChange(chord='Dm', timestamp=np.float64(0.3179591836734694)),
 ChordChange(chord='Gm7', timestamp=np.float64(1.2931972789115644)),
 ChordChange(chord='C#', timestamp=np.float64(2.291655328798186)),
 ChordChange(chord='Dm', timestamp=np.float64(3.3133333333333335)),
 ChordChange(chord='C#', timestamp=np.float64(4.335011337868481)),
 ChordChange(chord='Gm7', timestamp=np.float64(5.356689342403628)),
 ChordChange(chord='C#', timestamp=np.float64(6.378367346938776)),
 ChordChange(chord='Dm', timestamp=np.float64(7.400045351473923)),
 ChordChange(chord='A#7', timestamp=np.float64(9.443401360544218)),
 ChordChange(chord='Dm', timestamp=np.float64(10.465079365079365)),
 ChordChange(chord='Gm7', timestamp=np.float64(17.547165532879816)),
 ChordChange(chord='C#', timestamp=np.float64(18.568843537414963)),
 ChordChange(chord='Dm', timestamp=np.float64(19.59052154195011)),
 ChordChange(chord='Gm7', timestamp=np.float64(21.61065759637188)),
 ChordChange(chord='C#', timestamp=np.float64(22.60